# Discovering Exoplanets using XGboost

Duas principais razões para este notebook existir:

Obter prática num estudo ponta-a-ponta usando XGBoost
Lidar com dataset desbalanceado
Nosso estudo se concentrará em analisar dados de luz para predizer exoplanetas (planetas orbitando outras estrelas) no universo. Descobrir exoplanetas a partir da luz das estrelas requer a medição de flutuações de luz em intervalos de tempo prolongados. Como a mudança na luz geralmente é muito pequena, não é fácil determinar se um exoplaneta está realmente presente.

Este conjunto de dados de exoplanetas foi retirado do Telescópio Espacial Kepler da NASA, Campanha 3, verão de 2016. As informações sobre a fonte de dados estão disponíveis no Kaggle em [link](https://www.kaggle.com/datasets/keplersmachines/kepler-labelled-time-series-data). De todas as estrelas no conjunto de dados, 5.050 não possuem exoplanetas, enquanto 37 possuem exoplanetas.

Vamos Começar com um pequeno subconjunto dos dados

In [ ]:
import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
df = pd.read_csv('bases/exoplanets.csv', nrows=400)

In [ ]:
df.head()

O grande número de colunas (3198) listadas abaixo do DataFrame faz sentido. Ao procurar mudanças periódicas na luz, você precisa de pontos de dados suficientes para encontrar a periodicidade. As revoluções (movimento em torno do sol) dos planetas dentro do nosso próprio sistema solar variam de 88 dias (Mercúrio) a 165 anos (Netuno). Se os exoplanetas forem detectados, os pontos de dados devem ser examinados com frequência suficiente para não perder o trânsito do planeta quando o planeta orbitar na frente da estrela.

In [ ]:
df['LABEL'].value_counts()

A expectativa é que quando um exoplaneta bloquear a luz de uma estrela, o fluxo de luz diminua. Se as quedas no fluxo ocorrerem periodicamente, um exoplaneta é provavelmente o motivo, pois, por definição, um planeta é um objeto grande que orbita uma estrela.

Vamos visualizar os dados criando gráfico:

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
sns.set()

X = df.iloc[:,1:]
y = df.iloc[:,0]

def light_plot(index):
    y_vals = X.iloc[index]
    x_vals = np.arange(len(y_vals))
    plt.figure(figsize=(15,8))
    plt.xlabel('Number of Observations')
    plt.ylabel('Light Flux')
    plt.title('Light Plot ' + str(index), size=15)
    plt.plot(x_vals, y_vals)
    plt.show()

A função light_plot recebe como entrada o índice dos dados (a linha) que plota todos os pontos de dados como coordenadas y (o fluxo de luz) e o número de observações como coordenadas x.

In [ ]:
light_plot(0) # explaneta

In [ ]:
light_plot(37)# nao-exoplaneta

Há quedas claras nos dados, mas não são periódicas ao longo do gráfico. A frequência das quedas não se repete de forma consistente. Apenas esta evidência não é suficiente determinar a presença de um exoplaneta.

In [ ]:
light_plot(1) # exoplaneta

O gráfico mostra uma periodicidade clara com grandes quedas no fluxo de luz, tornando um exoplaneta extremamente provável! Se todos os gráficos fossem tão claros, o aprendizado de máquina seria desnecessário. Como os outros gráficos revelam, concluir que um exoplaneta está presente geralmente não é tão claro.

> Embora esse conjunto de dados seja uma série temporal, o objetivo não é prever o fluxo de luz para a próxima unidade de tempo, mas classificar a estrela com base em todos os dados. A este respeito, classificadores de aprendizado de máquina podem ser usados para prever se uma determinada estrela hospeda um exoplaneta. A ideia é treinar o classificador nos dados fornecidos, que por sua vez podem ser usados para prever exoplanetas em novos dados. Aqui, tentaremos classificar os exoplanetas dentro dos dados usando o XGBClassifier. Antes de passarmos para classificar os dados, devemos primeiro preparar os dados.

In [ ]:
df.info()

In [ ]:
# para cada coluna e, depois, para somar todas as colunas
df.isnull().sum().sum()

In [ ]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=2)

In [ ]:
model = XGBClassifier(booster='gbtree', objective='binary:logistic', random_state=2, verbosity=0)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
score = accuracy_score(y_pred, y_test)
print('Score: ' + str(score))

Pergunta: você obteve 89% de acurácia, mas os exoplanetas representam 10% dos dados, então como você sabe que seus resultados não são melhores do que um modelo que não prevê exoplanetas 100% do tempo?

> Aí está a questão. Se o modelo determinar que nenhuma estrela contém exoplanetas, sua acurácia será de aproximadamente 90%, pois 9 em cada 10 estrelas não contêm exoplanetas.

> com dados desbalanceados, acurácia não é suficiente

> podemos usar a matrix de confusao para analisar melhor o resultado

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)
# 86 nao-exoplanetas corretamente classificados e apenas 3 exoplanetas corretamente classificados

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

Ao escolher um método de score, é fundamental entender o objetivo. O objetivo no conjunto de dados Exoplanet é encontrar exoplanetas. Isso é óbvio. O que não é óbvio é como selecionar o melhor método de score para alcançar os resultados desejados.

Imagine dois cenários diferentes:

* Cenário 1: Das 4 estrelas de exoplanetas que o modelo de aprendizado de máquina prevê, 3 são na verdade estrelas de exoplanetas: precision = 3/4 = 75%.

* Cenário 2: Das 12 estrelas de exoplanetas, o modelo prevê corretamente 8 estrelas de exoplanetas. recall = 8/12 = 66%.

A resposta é que depende. O Recall é ideal para sinalizar possíveis casos positivos (exoplanetas) com o objetivo de encontrar todos eles. Precision é ideal para garantir que as previsões (exoplanetas) sejam realmente positivas.

Supondo que o objetivo do modelo de aprendizado de máquina seja encontrar o maior número possível de exoplanetas, o recall é uma excelente escolha. Por quê? Recall nos diz quantas das 12 estrelas de exoplanetas foram encontradas (2/12, 5/12, 12/12).

In [ ]:
from sklearn.metrics import recall_score
recall_score(y_test, y_pred, pos_label=2) #deixando claro que a classe positiva é a 2

## Tratando dados desbalanceados

Agora que temos um método de score apropriado para descobrir exoplanetas, é hora de explorar estratégias como resampling, undersampling e oversampling para corrigir os dados desequilibrados que causam a recall baixo.

* Uma estratégia para neutralizar dados desequilibrados é reamostrar os dados. É possível subamostrar os dados reduzindo as linhas da classe majoritária e superamostrando os dados repetindo as linhas da classe minoritária.

Vamos escrever uma função que nos permite subamostrar os dados por qualquer número de linhas. A função a seguir usa o XGBClassifier e o número de linhas como entrada e produz confusion matrix, classification report e recall score.

In [ ]:
def xgb_clf(model, nrows):

    df = pd.read_csv('bases/exoplanets.csv', nrows=nrows)
    # Split data into X and y
    X = df.iloc[:,1:]
    y = df.iloc[:,0]

    # Split data into train and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=2)

    # Fit xg_reg to training set
    model.fit(X_train, y_train)

    # Predict labels of test set, y_pred
    y_pred = model.predict(X_test)

    score = recall_score(y_test, y_pred, pos_label=2)
    
    print(confusion_matrix(y_test, y_pred))
    
    print(classification_report(y_test, y_pred))
        
    return score

In [ ]:
xgb_clf(XGBClassifier(random_state=2), nrows=800)

In [ ]:
xgb_clf(XGBClassifier(random_state=2), nrows=200)

In [ ]:
xgb_clf(XGBClassifier(random_state=2), nrows=74)

Outra técnica de reamostragem é a oversampling. Em vez de eliminar linhas, oversampling adiciona linhas copiando e redistribuindo os casos positivos.

Nossa estratégia é a seguinte:

* Crie um novo DataFrame que copie os casos positivos nove vezes.

* Concatene um novo DataFrame com o original para obter uma proporção de 10-10.

Antes de prosseguir, um aviso. Se os dados forem reamostrados antes de serem divididos em conjuntos de treinamento e teste, o recall será inflado.

Na reamostragem, serão feitas nove cópias dos casos positivos. Depois de dividir esses dados em conjuntos de treinamento e teste, as cópias provavelmente estarão contidas em ambos os conjuntos. Portanto, o conjunto de teste conterá a maioria dos mesmos pontos de dados que o conjunto de treinamento.

A estratégia apropriada é dividir os dados em um conjunto de treinamento e teste primeiro e depois reamostrar os dados:

In [ ]:
df_train = pd.merge(y_train, X_train, left_index=True, right_index=True)

In [ ]:
newdf = pd.DataFrame(np.repeat(df_train[df_train['LABEL']==2].values,9,axis=0))
newdf.columns = df_train.columns
df_train_resample = pd.concat([df_train, newdf])

df_train_resample['LABEL'].value_counts()

In [ ]:
X_train_resample = df_train_resample.iloc[:,1:]
y_train_resample = df_train_resample.iloc[:,0]

In [ ]:
# Instantiate the XGBRegressor, xg_reg
model = XGBClassifier(random_state=2, verbosity=0)

# Fit xg_reg to training set
model.fit(X_train_resample, y_train_resample)

# Predict labels of test set, y_pred
y_pred = model.predict(X_test)

score = recall_score(y_test, y_pred, pos_label=2)

print(confusion_matrix(y_test, y_pred))

print(classification_report(y_test, y_pred))

print(score)

Melhoramos um pouco o recall, mas aindaa longe do desejado. Vamos fazer um tunning dos hiperparametros do XGBoost

## Tuning XGBClassifier

vamos ajustar e dimensionar o XGBClassifier para obter o melhor valor de recall_score possível para o conjunto de dados Exoplanets. Primeiro, você ajustará os pesos usando scale_pos_weight e, em seguida, executará pesquisas de grade para encontrar a melhor combinação de hiperparâmetros. Além disso, você pontuará modelos para diferentes subconjuntos de dados antes de consolidar e analisar os resultados.

Scale_pos_weight é um hiperparâmetro usado para dimensionar o peso positivo. A ênfase aqui no positivo é importante porque o XGBoost assume que um valor alvo de 1 é positivo e um valor alvo de 0 é negativo.

No conjunto de dados Exoplanet, usamos o padrão 1 como negativo e 2 como positivo, conforme fornecido pelo conjunto de dados. Agora vamos mudar para 0 como negativo e 1 como positivo usando o método .replace().

In [ ]:
df['LABEL'] = df['LABEL'].replace(1, 0)
df['LABEL'] = df['LABEL'].replace(2, 1)

df['LABEL'].value_counts()

In [ ]:
# Split data into X and y
X = df.iloc[:,1:]
y = df.iloc[:,0]

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=2)

In [ ]:
model = XGBClassifier(scale_pos_weight=10, random_state=2,verbosity=0)

model.fit(X_train, y_train)

# Predict labels of test set, y_pred
y_pred = model.predict(X_test)

score = recall_score(y_test, y_pred)

print(confusion_matrix(y_test, y_pred))

print(classification_report(y_test, y_pred))

print(score) # o mesmo resultado que foi obtido usando oversampling

In [ ]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, StratifiedKFold, cross_val_score

In [ ]:
#baseline
kfold = StratifiedKFold(n_splits=2, shuffle=True, random_state=2)

model = XGBClassifier(scale_pos_weight=10, random_state=2,verbosity=0)

# Obtain scores of cross-validation
scores = cross_val_score(model, X, y, cv=kfold, scoring='recall')

# Display accuracy
print('Recall: ', scores)

# Display mean accuracy
print('Recall mean: ', scores.mean())

In [ ]:
def grid_search(params, random=False, X=X, y=y, model=XGBClassifier(random_state=2, scale_pos_weight=10,
                                                                   verbosity=0, use_label_encoder=False)): 
    
    xgb = model
    
    if random:
        grid = RandomizedSearchCV(xgb, params, cv=kfold, n_jobs=-1, random_state=2, scoring='recall')
    else:
        # Instantiate GridSearchCV as grid_reg
        grid = GridSearchCV(xgb, params, cv=kfold, n_jobs=-1, scoring='recall')
    
    # Fit grid_reg on X_train and y_train
    grid.fit(X, y)

    # Extract best params
    best_params = grid.best_params_

    # Print best params
    print("Best params:", best_params)
    
    # Compute best score
    best_score = grid.best_score_

    # Print best score
    print("Best score: {:.5f}".format(best_score))

In [ ]:
grid_search(params={'n_estimators':[50, 200, 400, 800]})

In [ ]:
grid_search(params={'learning_rate':[0.01, 0.05, 0.2, 0.3]})

In [ ]:
grid_search(params={'max_depth':[1, 2, 4, 8]})

In [ ]:
grid_search(params={'subsample':[0.3, 0.5, 0.7, 0.9]})

In [ ]:
grid_search(params={'gamma':[0.05, 0.1, 0.5, 1]})

In [ ]:
grid_search(params={'learning_rate':[0.001, 0.01, 0.03], 'max_depth':[1, 2], 'gamma':[0.025, 0.05, 0.5]})

In [ ]:
grid_search(params={'subsample':[0.3, 0.5, 0.7, 0.9, 1], 
                    'colsample_bylevel':[0.3, 0.5, 0.7, 0.9, 1], 
                    'colsample_bynode':[0.3, 0.5, 0.7, 0.9, 1], 
                    'colsample_bytree':[0.3, 0.5, 0.7, 0.9, 1]}, random=True)

In [ ]:
#migrando para o dataset perfeitameente balanceado
X_short = X.iloc[:74, :]
y_short = y.iloc[:74]

In [ ]:
grid_search(params={'max_depth':[1, 2, 3], 'colsample_bynode':[0.5, 0.75, 1]}, X=X_short, y=y_short, 
            model=XGBClassifier(random_state=2,verbosity=0,use_label_encoder=False))

In [ ]:
# all dataset
df_all = pd.read_csv('bases/exoplanets.csv')
df_all['LABEL'] = df_all['LABEL'].replace(1, 0)
df_all['LABEL'] = df_all['LABEL'].replace(2, 1)

X_all = df_all.iloc[:,1:]
y_all = df_all.iloc[:,0]

In [ ]:
df_all['LABEL'].value_counts()

In [ ]:
weight = int(5050/37)

In [ ]:
model = XGBClassifier(scale_pos_weight=weight, random_state=2,verbosity=0)

# Obtain scores of cross-validation
scores = cross_val_score(model, X_all, y_all, cv=kfold, scoring='recall')

# Display accuracy
print('Recall:', scores)

# Display mean accuracy
print('Recall mean:', scores.mean())

In [ ]:
grid_search(params={'learning_rate':[0.001, 0.01]}, X=X_all, y=y_all, model=XGBClassifier(scale_pos_weight=weight, 
                                                                                          random_state=2,
                                                                                         verbosity=0,
                                                                                         use_label_encoder=False))

In [ ]:
grid_search(params={'max_depth':[1, 2],'learning_rate':[0.001]}, X=X_all, y=y_all, 
            model=XGBClassifier(scale_pos_weight=weight, random_state=2,verbosity=0,use_label_encoder=False))

In [ ]:
def final_model(X, y, model):
    model.fit(X, y)
    y_pred = model.predict(X_all)
    score = recall_score(y_all, y_pred)
    print(score)
    print(confusion_matrix(y_all, y_pred))
    print(classification_report(y_all, y_pred))

In [ ]:
# balanceado = 74 linhas
final_model(X_short, y_short, XGBClassifier(max_depth=2, colsample_by_node=0.5, random_state=2,
                                            verbosity=0,use_label_encoder=False))

In [ ]:
# 400 linhas
final_model(X, y, XGBClassifier(max_depth=2, colsample_bynode=0.5, scale_pos_weight=10, random_state=2,
                               verbosity=0,use_label_encoder=False))

In [ ]:
# todas as linhas
final_model(X_all, y_all, XGBClassifier(max_depth=2, colsample_bynode=0.5, 
                                        scale_pos_weight=weight, 
                                        random_state=2,
                                       verbosity=0,use_label_encoder=False))